In [ ]:
from typing import Tuple
import numpy as np

def generate_grid_mesh(height: int, width: int, flatten_arrays: bool = True) -> Tuple[np.ndarray, np.ndarray]:
  # define grid offsets
  grid_offsets = np.zeros((height + 1, width + 1, 2), dtype=np.int32)
  grid_offsets[:, :, 0] = np.arange(height + 1)[:, None]
  grid_offsets[:, :, 1] = np.arange(width + 1)[None, :]
  
  unique_grid_unit_points = np.array([[0.0, 0.0], [0.5, 0.0], [0.0, 0.5], [0.5, 0.5]])

  # h+1 x w+1 x 4 x 2
  grid_points = unique_grid_unit_points[None, None, :] + np.flip(grid_offsets, -1)[:, :, None, :]

  grid_indices = np.zeros((height + 1, width + 1, 4, 3), dtype=np.int32)
  grid_indices[:, :, :, 0] = np.arange(4)[None, None, :]
  grid_indices[:, :, :, 1:] = grid_offsets[:, :, None, :]

  # h x w x 4 x 3
  a = grid_indices[:-1, :-1]
  b = grid_indices[:-1, 1:]
  c = grid_indices[1:, :-1]
  d = grid_indices[1:, 1:]
  gridded_triangles = [
    (a[:, :, 0], a[:, :, 3], a[:, :, 1]),
    (a[:, :, 0], a[:, :, 2], a[:, :, 3]),
    (a[:, :, 1], a[:, :, 3], b[:, :, 0]),
    (a[:, :, 3], b[:, :, 2], b[:, :, 0]),
    (a[:, :, 2], c[:, :, 0], a[:, :, 3]),
    (a[:, :, 3], c[:, :, 0], c[:, :, 1]),
    (a[:, :, 3], c[:, :, 1], d[:, :, 0]),
    (a[:, :, 3], d[:, :, 0], b[:, :, 2])
  ]
  triangles = []
  for pointset in gridded_triangles:
    # h x w x 3 x 3 (third axis is the grid coordinate)
    pointset = np.stack(pointset, axis=-1)
    # h x w x 3 (grid coordinate converted to flat index)
    pointset = (
      pointset[:, :, 2, :] * 4
      + pointset[:, :, 1, :] * (width + 1) * 4
      + pointset[:, :, 0, :]
    )
    triangles.append(pointset)
  triangles = np.stack(triangles)

  points = np.concatenate((grid_points, np.zeros((height + 1, width + 1, 4, 1))), axis=-1)
  if flatten_arrays:
    points = points.reshape(-1, 3)
    # (8 x h x w) x 3 (list of triangles - 8 per grid face)
    triangles = np.concatenate(triangles).reshape(-1, 3)
  return points, triangles

points, triangles = generate_grid_mesh(4, 2)
save_stl(points, triangles, 'output.stl')

In [ ]:
from stl import mesh

def save_stl(vertices, faces, filename):
    """
    Save vertex and face data as an STL file.

    @param vertices: n x 3 array of 3D points.
    @param faces: m x 3 array of vertex indices specifying triangles.
    """
    # Create the mesh
    triangle_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    
    # Fill in the mesh vertices for each triangle
    for i, face in enumerate(faces):
        for j in range(3):
            triangle_mesh.vectors[i][j] = vertices[face[j]]
    
    # Save to file
    triangle_mesh.save(filename)

# Example usage
vertices = np.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

faces = np.array([
    [0, 1, 2],
    [0, 1, 3],
    [0, 2, 3],
    [1, 2, 3]
])

save_stl(points, triangles, 'output.stl')

In [203]:
import os
os.chdir('C:/Users/clack/Projects/nwm')

from experiment.nav2d import Topo

state = Topo.random()
terrain = state.terrain[64:256-64, 64:256-64]
# h+1 x w+1 x 4 x 3
height = terrain.shape[0] * 2 - 1
width = terrain.shape[1] * 2 - 1
points, triangles = generate_grid_mesh(height, width, flatten_arrays=False)
grid_offsets = np.zeros((height + 1, width + 1, 2), dtype=np.int32)
grid_offsets[:, :, 0] = np.arange(height + 1)[:, None]
grid_offsets[:, :, 1] = np.arange(width + 1)[None, :]

even_x = grid_offsets[:, :, 0] % 2 == 0
even_y = grid_offsets[:, :, 1] % 2 == 0
unit_mask = even_x & even_y
y_sandwich_mask = ~even_x & even_y & (grid_offsets[:, :, 0] < height)
x_sandwich_mask = even_x & ~even_y & (grid_offsets[:, :, 1] < width)
corner_mask = ~even_x & ~even_y & (grid_offsets[:, :, 0] < height) & (grid_offsets[:, :, 1] < width)

modulated_terrain_height = np.zeros_like(terrain)
modulated_terrain_height[terrain < state.wall_height] = terrain[terrain < state.wall_height] * 10
modulated_terrain_height[terrain >= state.wall_height] = terrain[terrain >= state.wall_height] * 20 + 4

# set every other 3x3 grid region to the corresponding terrain height
points[unit_mask, :, 2] = modulated_terrain_height[:, :, None].repeat(4, -1).reshape(-1, 4)
points[y_sandwich_mask, 0, 2] = modulated_terrain_height[:-1, :].reshape(-1)
points[y_sandwich_mask, 1, 2] = modulated_terrain_height[:-1, :].reshape(-1)
points[x_sandwich_mask, 0, 2] = modulated_terrain_height[:, :-1].reshape(-1)
points[x_sandwich_mask, 2, 2] = modulated_terrain_height[:, :-1].reshape(-1)
points[corner_mask, 0, 2] = modulated_terrain_height[:-1, :-1].reshape(-1)

# set interstitial points to interpolated heights
y_interpolated_height = (modulated_terrain_height[:-1] + modulated_terrain_height[1:]) / 2
points[y_sandwich_mask, 2, 2] = y_interpolated_height.reshape(-1)
points[y_sandwich_mask, 3, 2] = y_interpolated_height.reshape(-1)
points[corner_mask, 2, 2] = y_interpolated_height[:, :-1].reshape(-1)
x_interpolated_height = (modulated_terrain_height[:, :-1] + modulated_terrain_height[:, 1:]) / 2
points[x_sandwich_mask, 1, 2] = x_interpolated_height.reshape(-1)
points[x_sandwich_mask, 3, 2] = x_interpolated_height.reshape(-1)
points[corner_mask, 1, 2] = x_interpolated_height[:-1, :].reshape(-1)
corner_interpolated_height = (x_interpolated_height[:-1] + x_interpolated_height[1:] + y_interpolated_height[:, :-1] + y_interpolated_height[:, 1:]) / 4
points[corner_mask, 3, 2] = corner_interpolated_height.reshape(-1)

colors = np.array([
  [0.6, 0.5, 0.4],  # dirt paths
  [0.25, 0.3, 0.2]  # green mountaintops
])
colormap = np.zeros((8, height + 1, width + 1), dtype=np.int32)

# modulated_terrain_height = np.zeros_like(terrain, )
# modulated_terrain_height[terrain < state.wall_height] = terrain[terrain < state.wall_height] * 10
# modulated_terrain_height[terrain >= state.wall_height] = terrain[terrain >= state.wall_height] * 20 + 4
# colormap[:, unit_mask] = (terrain >= state.wall_height).reshape(1, -1)
texture = colors[(terrain.transpose(0, 1) >= state.wall_height).astype(np.int32)]
texture_coordinates = points[:, :, :, :2].copy().reshape(-1, 2)
texture_coordinates[:, 0] = texture_coordinates[:, 0] / (height + 1)
texture_coordinates[:, 1] = 1 - texture_coordinates[:, 1] / (width + 1)

save_stl(points.reshape(-1, 3), triangles.reshape(-1, 3), 'output.stl')
create_textured_mesh(points.reshape(-1, 3), triangles.reshape(-1, 3), texture_coordinates, texture, 'terrain.obj', 'terrain.png')

Created terrain.obj, terrain.mtl, and terrain.png
Vertices: 262144
Faces: 520200
Texture size: 128x128


In [197]:
import numpy as np
from PIL import Image

def create_textured_mesh(vertices, faces, texture_coords, texture_image,
                         obj_filename, texture_filename):
    """
    Create a mesh with arbitrary UV texture mapping.
    
    Parameters:
    -----------
    vertices : np.ndarray, shape (n, 3)
        Vertex positions
    faces : np.ndarray, shape (m, 3)
        Triangle indices into vertices
    texture_coords : np.ndarray, shape (n, 2)
        UV coordinates for each vertex, range [0, 1]
        (u, v) where u is horizontal, v is vertical
    texture_image : np.ndarray, shape (h, w, 3)
        RGB texture image, values in range [0, 255] or [0, 1]
    obj_filename : str
        Output OBJ filename
    texture_filename : str
        Output PNG texture filename
    """
    
    # Ensure texture is in [0, 255] range
    if texture_image.max() <= 1.0:
        texture_image = (texture_image * 255).astype(np.uint8)
    else:
        texture_image = texture_image.astype(np.uint8)
    
    # Save texture image
    Image.fromarray(texture_image).save(texture_filename)
    
    # Write OBJ file
    with open(obj_filename, 'w') as f:
        f.write("# OBJ file with custom UV texture mapping\n")
        f.write(f"mtllib {obj_filename.replace('.obj', '.mtl')}\n\n")
        
        # Write vertices
        for vert in vertices:
            f.write(f"v {vert[0]} {vert[1]} {vert[2]}\n")
        
        f.write("\n")
        
        # Write UV coordinates
        for uv in texture_coords:
            f.write(f"vt {uv[0]} {uv[1]}\n")
        
        f.write("\n")
        f.write("usemtl material_0\n")
        
        # Write faces (OBJ uses 1-based indexing)
        for face in faces:
            # Each vertex index also refers to its UV coordinate (same index)
            f.write(f"f {face[0]+1}/{face[0]+1} {face[2]+1}/{face[2]+1} {face[1]+1}/{face[1]+1}\n")
            # f.write(f"f {face[0]+1}/{face[0]+1} {face[1]+1}/{face[1]+1} {face[2]+1}/{face[2]+1}\n")
    
    # Write MTL file
    mtl_filename = obj_filename.replace('.obj', '.mtl')
    with open(mtl_filename, 'w') as f:
        f.write("# MTL file\n")
        f.write("newmtl material_0\n")
        f.write("Ka 1.0 1.0 1.0\n")
        f.write("Kd 1.0 1.0 1.0\n")
        f.write("Ks 0.0 0.0 0.0\n")
        f.write(f"map_Kd {texture_filename}\n")
    
    print(f"Created {obj_filename}, {mtl_filename}, and {texture_filename}")
    print(f"Vertices: {len(vertices)}")
    print(f"Faces: {len(faces)}")
    print(f"Texture size: {texture_image.shape[0]}x{texture_image.shape[1]}")